In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
from tbdynamics.calibration.utils import (
    get_bcm,
    calculate_covid_diff_quantiles,
)
import arviz as az
from tbdynamics.inputs import load_targets
from tbdynamics.constants import  quantiles
from estival.sampling import tools as esamp
import xarray as xr

In [ ]:
OUT_PATH = Path.cwd() / 'runs/best2208'
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [ ]:
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))

In [ ]:
az.to_netcdf(inference_data, 'inference_data.nc')

In [ ]:
loaded_inference_data = az.from_netcdf('inference_data.nc')

In [ ]:
loaded_inference_data

In [ ]:
extracted_dataset = xr.Dataset(idata_extract)
extracted_idata = az.InferenceData(posterior=extracted_dataset)

In [ ]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [ ]:
quantile_ouputs_diff = calculate_covid_diff_quantiles(params, extracted_idata, ['cumulative_diseased', 'cumulative_deaths'],[2021.0,2022.0],1)


In [ ]:
quantile_ouputs_diff